In [6]:
from flask import Flask, request
import requests
import json

app = Flask(__name__)

# Trendyol API bilgilerini buraya girin
api_key = "YOUR_API_KEY"
supplier_id = "YOUR_SUPPLIER_ID"

# Webhook endpoint'i
@app.route("/webhook", methods=["POST"])
def handle_webhook():
    data = request.json
    
    # Alışveriş bildirimi olduğunda stok güncelleme işlemi yap
    if data["eventType"] == "OrderCreated":
        order_id = data["data"]["orderId"]
        products = data["data"]["orderItems"]
        
        for product in products:
            barcode = product["barcode"]
            quantity_change = -1  # Stok düşürme için -1 değerini kullanabilirsiniz
            
            update_stock(barcode, quantity_change, order_id)
        
        invoice_id = data["data"]["invoiceId"]
        invoice_data = get_invoice_data(invoice_id)
        
        if invoice_data:
            create_invoice(invoice_data, order_id)
        
        return "Webhook işlendi.", 200
    
    return "Geçersiz bildirim.", 400

# Trendyol API'sine stok güncelleme isteği gönderme
def update_stock(barcode, quantity_change, order_id):
    url = f"https://api.trendyol.com/sapigw/suppliers/{supplier_id}/v2/products/{barcode}/stock"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Basic {api_key}"
    }

    payload = {
        "quantity": quantity_change
    }

    response = requests.put(url, headers=headers, data=json.dumps(payload))

    if response.status_code == 200:
        print(f"Stok güncelleme başarılı: Order ID - {order_id}")
    else:
        print(f"Stok güncelleme hatası: Order ID - {order_id}")
        print(f"HTTP Hata Kodu: {response.status_code}")
        print(f"API Yanıtı: {response.text}")

# Trendyol API'sinden fatura bilgilerini alma
def get_invoice_data(invoice_id):
    url = f"https://api.trendyol.com/sapigw/invoicing/{invoice_id}"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Basic {api_key}"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        invoice_data = response.json()
        return invoice_data
    else:
        print(f"Fatura bilgisi alınamadı: Invoice ID - {invoice_id}")
        print(f"HTTP Hata Kodu: {response.status_code}")
        print(f"API Yanıtı: {response.text}")
        return None

# Trendyol API'sine fatura oluşturma isteği gönderme
def create_invoice(invoice_data, order_id):
    url = f"https://api.trendyol.com/sapigw/suppliers/{supplier_id}/v2/invoices"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Basic {api_key}"
    }

    payload = {
        "invoiceData": invoice_data
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))
    
    if response.status_code == 200:
        print(f"Fatura oluşturma başarılı: Order ID - {order_id}")
    else:
        print(f"Fatura oluşturma hatası: Order ID - {order_id}")
        print(f"HTTP Hata Kodu: {response.status_code}")
        print(f"API Yanıtı: {response.text}")

# Trendyol API'sine ürün fiyatı güncelleme isteği gönderme
def update_price(barcode, new_price):
    url = f"https://api.trendyol.com/sapigw/suppliers/{supplier_id}/v2/products/{barcode}/prices"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Basic {api_key}"
    }

    payload = {
        "price": new_price
    }

    response = requests.put(url, headers=headers, data=json.dumps(payload))

    if response.status_code == 200:
        print(f"Fiyat güncelleme başarılı: Barcode - {barcode}")
    else:
        print(f"Fiyat güncelleme hatası: Barcode - {barcode}")
        print(f"HTTP Hata Kodu: {response.status_code}")
        print(f"API Yanıtı: {response.text}")

# Trendyol API'sine ürün stok sayısını artırma isteği gönderme
def increase_stock(barcode, quantity_increase):
    url = f"https://api.trendyol.com/sapigw/suppliers/{supplier_id}/v2/products/{barcode}/stock"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Basic {api_key}"
    }

    payload = {
        "quantity": quantity_increase
    }

    response = requests.put(url, headers=headers, data=json.dumps(payload))

    if response.status_code == 200:
        print(f"Stok artırma başarılı: Barcode - {barcode}")
    else:
        print(f"Stok artırma hatası: Barcode - {barcode}")
        print(f"HTTP Hata Kodu: {response.status_code}")
        print(f"API Yanıtı: {response.text}")


In [2]:
import requests

url = 'https://api.trendyol.com/sapigw/sellerreturnservice/v2/returns'  # İade talepleri API endpointi
headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer <access_token>'  # Trendyol API erişim belirteci (access token)
}

payload = {
    'returnItems': [
        {
            'orderLineItemId': '<order_line_item_id>',  # İade yapılacak ürünün sipariş satırı kimliği
            'quantity': 1,  # İade miktarı
            'returnReasonId': '<return_reason_id>',  # İade nedeni kimliği
            'customerComment': 'İade nedeni açıklaması'  # İade açıklaması
        }
    ]
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 201:
    print('İade talebi başarıyla oluşturuldu.')
else:
    print('İade talebi oluşturulurken bir hata oluştu. Hata kodu:', response.status_code)
    print('Hata mesajı:', response.text)
    
url = 'https://api.trendyol.com/sapigw/sellerreturnservice/v2/returns/<return_id>'  # İade talebi API endpointi
headers = {
    'Authorization': 'Bearer <access_token>'  # Trendyol API erişim belirteci (access token)
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    return_data = response.json()
    return_status = return_data['data']['status']
    print('İade talebi durumu:', return_status)
else:
    print('İade talebi durumu alınır')

İade talebi oluşturulurken bir hata oluştu. Hata kodu: 503
Hata mesajı: <html><body><b>Http/1.1 Service Unavailable</b></body> </html>
İade talebi durumu alınır


In [4]:
import requests

API_URL = "https://api.trendyol.com/sapigw/seller/orders/{orderNumber}/package-operations"  # Trendyol API URL'sini buraya yerleştirin

# Trendyol API erişim bilgilerinizi buraya girin
API_KEY = "TRENDYOL_API_KEY"
SECRET_KEY = "TRENDYOL_SECRET_KEY"

def get_tracking_info(order_number):
    headers = {
        "Authorization": f"Basic {API_KEY}:{SECRET_KEY}",
        "Content-Type": "application/json"
    }

    url = API_URL.format(orderNumber=order_number)
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        tracking_info = response.json()
        return tracking_info
    else:
        print("Kargo takip bilgileri alınamadı.")
        return None

def track_order():
    order_number = input("Takip etmek istediğiniz sipariş numarasını girin: ")

    tracking_info = get_tracking_info(order_number)

    if tracking_info:
        print("\n--- Kargo Takip Bilgileri ---")
        print(f"Sipariş Numarası: {tracking_info['orderNumber']}")
        print(f"Kargo Takip Numarası: {tracking_info['trackingNumber']}")
        print(f"Kargo Şirketi: {tracking_info['cargoCompany']['name']}")
        print(f"Gönderi Durumu: {tracking_info['cargoTrackingStatus']}")
        print("")

track_order()

Takip etmek istediğiniz sipariş numarasını girin: 12321321
Kargo takip bilgileri alınamadı.
